In [1]:
import os, glob
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")

from tools.utils import files_setup as fs
from tools.model_tools.network_parsers.shallow_net import Shallow_CNN
from tools.model_tools.network_parsers.deep_net import Deep_CNN
from tools.analytical_tools.matrix_tools.confounds import create_confound_matrix, context_confound_submat, category_confound_submat
from tools.analytical_tools.matrix_tools.ratios_and_stats import ratios_and_pvalues, context_category_pairwise_ttest

from constants import OUTPUT_MODELS_PATH, RAW_CONTEXT_RATIOS_FILE, RAW_CATEGORY_RATIOS_FILE, CONTEXT_EXEMPLARS, CATEGORY_EXEMPLARS, CONTEXTS, CATEGORIES, SHALLOW_MODEL, DEEP_MODEL, COL_NAMES, DATA_PATH, DIRECTORIES_FOR_ANALYSIS, END_FILE_NUMBER

In [3]:
ScenesConfData = pd.read_excel("./stimuli_with_scenes.xlsx")
ScenesConfData

,Context,Object 1,Object 2,Scene 1,Overlap with Obj 1?,Overlap with Obj 2?,Condition,Unnamed: 7,Unnamed: 8,confounding contexts
0,12,birdcage,parrot,?,NaN,NaN,no scene,NaN,NaN,NaN
1,13,birdhouse,bird,?,NaN,NaN,no scene,NaN,NaN,NaN
2,22,chess board,chess piece,?,NaN,NaN,no scene,NaN,NaN,NaN
3,30,ambulance,stretcher,?,NaN,NaN,no scene,NaN,NaN,NaN
4,33,fish,fishbowl,?,NaN,NaN,no scene,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
68,6,basketball,rim,court,no,yes,overlapping with one object,NaN,NaN,NaN
69,10,bed,dresser,bedroom,yes,no,overlapping with one object,NaN,NaN,do not compare with context 7
70,1,Easel,Pallette,Art Studio,yes,yes,Scene Overlapping with both,NaN,NaN,NaN
71,47,laundry,washer,laundryroom,yes,yes,Scene Overlapping with both,NaN,NaN,NaN


In [8]:
ScenesConfDataNew = ScenesConfData.loc[ScenesConfData["confounding contexts"].notna()]
ScenesConfDataNew

,Context,Object 1,Object 2,Scene 1,Overlap with Obj 1?,Overlap with Obj 2?,Condition,Unnamed: 7,Unnamed: 8,confounding contexts
13,7,bathrobe,slippers,bedroom,no,no,non-overlapping,NaN,NaN,do not compare with context 10
14,8,beachball,pail,beach,no,no,non-overlapping,NaN,NaN,"do not compare with context 9, 53, 14"
15,9,beach umbrella,chair,beach,no,no,non-overlapping,NaN,NaN,"do not compare with context 8, 53, 14"
16,11,bike,helmet,street,no,no,non-overlapping,NaN,NaN,"do not compare with context 60, 61"
17,14,anchor,helm,boat,no,no,non-overlapping,NaN,NaN,"do not compare with context: 8, 9, 53"
20,17,burger,fries,diner,no,no,non-overlapping,NaN,NaN,"do not compare with context, 57, 59, 23"
22,19,carpet,vacuum,living room,no,no,non-overlapping,NaN,NaN,"do not compare with context 48, 49, 70"
24,21,cheese,cheese grater,kitchen,no,no,non-overlapping,NaN,NaN,"do not compare with context 27, 45, 46, 29"
25,23,high chair,sippy cup,dining room/kitchen,no,no,non-overlapping,NaN,NaN,"do not compare with context, 57, 59, 17"
26,25,belt,pants,closet,no,no,non-overlapping,NaN,NaN,do not compare with context 26


In [22]:
# get all the numbers from a string
pattern = re.compile(r"\d\d?")

ScenesConfDataNew["CONFOUNDS"] =  ScenesConfDataNew["confounding contexts"].transform(lambda x : [int(num) for num in re.findall(pattern,x)])

In [25]:
ScenesConfDataNew = ScenesConfDataNew.reset_index(drop=True)
ScenesConfDataNew

,Context,Object 1,Object 2,Scene 1,Overlap with Obj 1?,Overlap with Obj 2?,Condition,Unnamed: 7,Unnamed: 8,confounding contexts,CONFOUNDS
0,7,bathrobe,slippers,bedroom,no,no,non-overlapping,NaN,NaN,do not compare with context 10,[10]
1,8,beachball,pail,beach,no,no,non-overlapping,NaN,NaN,"do not compare with context 9, 53, 14","[9, 53, 14]"
2,9,beach umbrella,chair,beach,no,no,non-overlapping,NaN,NaN,"do not compare with context 8, 53, 14","[8, 53, 14]"
3,11,bike,helmet,street,no,no,non-overlapping,NaN,NaN,"do not compare with context 60, 61","[60, 61]"
4,14,anchor,helm,boat,no,no,non-overlapping,NaN,NaN,"do not compare with context: 8, 9, 53","[8, 9, 53]"
5,17,burger,fries,diner,no,no,non-overlapping,NaN,NaN,"do not compare with context, 57, 59, 23","[57, 59, 23]"
6,19,carpet,vacuum,living room,no,no,non-overlapping,NaN,NaN,"do not compare with context 48, 49, 70","[48, 49, 70]"
7,21,cheese,cheese grater,kitchen,no,no,non-overlapping,NaN,NaN,"do not compare with context 27, 45, 46, 29","[27, 45, 46, 29]"
8,23,high chair,sippy cup,dining room/kitchen,no,no,non-overlapping,NaN,NaN,"do not compare with context, 57, 59, 17","[57, 59, 17]"
9,25,belt,pants,closet,no,no,non-overlapping,NaN,NaN,do not compare with context 26,[26]


In [27]:
confounds = dict()
for c1, c2  in zip(ScenesConfDataNew.Context, ScenesConfDataNew.CONFOUNDS):
    confounds[c1] = c2

In [43]:
context_pattern = re.compile(r"\d\d")
contexts = list()
for string in DIRECTORIES_FOR_ANALYSIS:
    
    match = re.search(context_pattern, string)
    if match:
        contexts.append(int(string[match.start():match.end()]))
    else:
        # no match found ...
        print("no match found")

In [59]:
53 in contexts

True

In [62]:
context_indices = dict(enumerate(contexts))
context_indices = {v:k for k,v in context_indices.items()}

In [80]:
confound_matrix = np.eye(50,50)

In [86]:
for key, vals in confounds.items():

    print(key,vals)
    for v in vals:

        try:
            print(context_indices[key], context_indices[v])
            confound_matrix[context_indices[key], context_indices[v]] = 1
        except:
            print("error in index", key, v)
    print("*"*10)


7 [10]
error in index 7 10
**********
8 [9, 53, 14]
2 3
2 33
error in index 8 14
**********
9 [8, 53, 14]
3 2
3 33
error in index 9 14
**********
11 [60, 61]
4 38
4 39
**********
14 [8, 9, 53]
error in index 14 8
error in index 14 9
error in index 14 53
**********
17 [57, 59, 23]
7 36
7 37
7 12
**********
19 [48, 49, 70]
9 31
9 32
9 48
**********
21 [27, 45, 46, 29]
11 15
11 29
11 30
11 17
**********
23 [57, 59, 17]
12 36
12 37
12 7
**********
25 [26]
13 14
**********
26 [25]
14 13
**********
27 [21, 45, 46, 29]
15 11
15 29
15 30
15 17
**********
29 [21, 45, 46, 27]
17 11
17 29
17 30
17 15
**********
31 [42, 32]
18 26
18 19
**********
32 [42, 31]
19 26
19 18
**********
42 [31, 32]
26 18
26 19
**********
45 [21, 29, 46, 27]
29 11
29 17
29 30
29 15
**********
46 [21, 45, 29, 27]
30 11
30 29
30 17
30 15
**********
48 [19, 49, 70]
31 9
31 32
31 48
**********
49 [48, 19, 70]
32 31
32 9
32 48
**********
53 [8, 9, 14]
33 2
33 3
error in index 53 14
**********
54 [55]
34 35
**********
55 [54]


In [95]:
confound_matrix[48,31]

1.0

In [96]:
# now save this context matrix as npy file
np.save("./confounding_data/scenes_obj/context_confounds.npy",confound_matrix)

In [104]:
np.savetxt("confounding_data/Scenes_Obj/context_confounds.txt",confound_matrix)

In [7]:
np.load("confounding_data/scenes_obj/context_confounds.npy")[48,31]

1.0

In [4]:
from constants import *

In [8]:
CONTEXT_CONFOUNDS == "confounding_data/scenes_obj/context_confounds.npy"

False

In [9]:
CONTEXT_CONFOUNDS

'confouding_data/scenes_obj/context_confounds.npy'

In [6]:
np.load(CONTEXT_CONFOUNDS)

FileNotFoundError: [Errno 2] No such file or directory: 'confouding_data/scenes_obj/context_confounds.npy'

In [99]:
DATA_PATH

'./data/scenes_obj/'

In [ ]:
'confouding_data/scenes_obj/context_confounds.npy'